# [ 데이터 수집 과제 제출 ]
- 스타벅스 사이트 -> 지점 정보 추출
- 데이터의 최종 형태[{'지점명':'GS타워','주소':'서울특별시 강남구 논현로 508 (역삼동)','시군구':'강남구','시도':'서울특별시','타입':'R|G|DT'},...]
- 이런 내용이 나오는 "데이터수집_본인이름.ipynb"  노트 파일 제출
- 3월 10일 24시까지 제출
- 제출 위치는 추후 공지

In [8]:
# 모듈 불러오기

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

import time

import random

from tqdm import tqdm

In [9]:
# 1. 크롬 창 열기
# 경로 상에 문제가 있는 것으로 생각되어서 webdriver_manager를 통해 크롬창을 열기

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [10]:
# 2. 스타벅스 맵 페이지에 접근하기

TARGET_SITE = 'https://www.starbucks.co.kr/store/store_map.do'
driver.get( TARGET_SITE )
time.sleep(2)

In [11]:
# 3. 시/도의 후보값
# 시/도에 접근하기 위해 시/도의 요소들을 찾아주기

sido_opts = driver.find_elements(By.CSS_SELECTOR, '.sido_arae_box > li > a')

In [12]:
# 함수 만들기

# 조건문을 안에서 사용하지않고 밖에 함수를 만들어주어 최대한 간결하게 표현해보기
def find_loc(tmp):
    if tmp.get_attribute('class') == 'pin_general':
        loc_type = 'G'
    elif tmp.get_attribute('class') == 'pin_reserve':
        loc_type = 'R'
    elif tmp.get_attribute('class') == 'pin_generalDT':
        loc_type = 'DT'
    elif tmp.get_attribute('class') == 'pin_generalWT':
        loc_type = 'WT'
    return loc_type

# 중복되는 긴 함수를 간결하게 표현하기

exchange_text = lambda selector : driver.find_element(By.CSS_SELECTOR, selector)
exchange_texts = lambda selector : driver.find_elements(By.CSS_SELECTOR, selector)

In [13]:
# ****스크롤을 내리는 행위를 할 때 전체화면으로 띄워놓고 코드를 실행해야 온전한 결과를 생성 가능****
# 창이 작아지면 그에 따라 보여지는 지역 검색 창도 작아져서 코드가 실행이 되지않는다.
# 4-1 지역 검색 버튼에 접근
# 4-2 원하는 시/구에 접근
# 4-3 각 시/구에서 전체 버튼에 접근

# 매장 정보들을 담아줄 빈 리스트 생성
starbucks_loc = []

# 매장 정보를 잘 불러왔는지 확인하기 위해서 전체 검색 결과 옆의 숫자를 count해준다.
count = 0

for cur_sido in tqdm(range(len(sido_opts))):
    
    # 각 행동마다 랜덤 시간을 부여하자

    # *세종시는 전체 버튼이 없이 바로 접근이 가능하다*
    # 인덱스의 마지막 값이 세종
    if cur_sido == len(sido_opts) - 1:
        
        # 4-1
        # 우선 지역 검색 창으로 이동을 해야 시/도의 정보를 확인할 수 있으므로 클릭하기
        # 어떤 상황에서도 검색 창을 초기화할 수 도 있다.
        exchange_text('.loca_search > h3 > a').click()
        time.sleep(random.randint(1,3))
        
        # 4-2
        # 각 시/도에 클릭을 해야 전체, 시군구 정보에 접근이 가능하다.
        exchange_text(f'.sido_arae_box > li:nth-child({cur_sido + 1}) > a' ).click()
        time.sleep(random.randint(7,10))
        
        # 전체 버튼 클릭 없이 총 매장 정보에 바로 접근
        sigun = exchange_texts('.quickSearchResultBoxSidoGugun > li')
        loc_count = int( exchange_text('div.loca_step3 > div.result_num_wrap > span').text)
        count += loc_count
        
        # 시군구 전체에 해당하는 값을 돌면서 딕셔너리 정보를 리스트에 append 
        for cur_sigun in range(len(sigun)):
            tmp = exchange_text(f'.quickSearchResultBoxSidoGugun> li:nth-child({cur_sigun+1}) > i')
            starbucks_loc.append(
                {
                    # 함수를 사용하여 코드를 간결하게 해보기
                    '지점명'   :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > strong').text.strip(),
                    '주소'     :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > p').text.strip().split('\n')[0],
                    '시도'     :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > p').text.strip().split()[0],
                    '시군구'   :  '시군구 정보 없음',
                    '타입'     :  find_loc(tmp)
                }
            )
            # append 후에 지역 검색 창의 스크롤을 내려줘야 정보가 담긴다.
            # 해당 매장에 관한 정보가 담기면 다음 2번째의 매장으로 스크롤 되는 코드
            if cur_sigun < len(sigun) - 2:
                action = ActionChains(driver)
                scrollList = exchange_texts(f'.quickSearchResultBoxSidoGugun> li')
                action.move_to_element(scrollList[cur_sigun+2]).perform()
            else:
                 continue
            #time.sleep(random.randint(1,2))
    
    else:
        # 4-1
        # 우선 지역 검색 창으로 이동을 해야 시/도의 정보를 확인할 수 있으므로 클릭하기
        # 어떤 상황에서도 검색 창을 초기화할 수 도 있다.
        exchange_text('.loca_search > h3 > a').click()
        time.sleep(random.randint(1,3))
        
        # 4-2
        # 각 시/도에 클릭을 해야 전체, 시군구 정보에 접근이 가능하다.
        exchange_text(f'.sido_arae_box > li:nth-child({cur_sido + 1}) > a' ).click()
        time.sleep(random.randint(1,3))
        
        # 4-3
        # 전체 버튼을 눌러 각 시/도의 매장 정보를 확인할 수 있다.
        exchange_text('.gugun_arae_box > li:nth-child(1) > a').click()
        time.sleep(random.randint(7,10))
        
        # 5-1 각 시/도의 매장수를 파악하여 반복문 안에서 진행
        sigun = exchange_texts('.quickSearchResultBoxSidoGugun > li')
        loc_count = int(exchange_text('div.loca_step3 > div.result_num_wrap > span').text)
        count += loc_count
        
        
        # 5-2 추가적인 반복문을 통해서 각 시/도의 매장 정보를 담기
        
        for cur_sigun in range(len(sigun)):
            tmp = exchange_text(f'.quickSearchResultBoxSidoGugun> li:nth-child({cur_sigun+1}) > i')
            starbucks_loc.append(
                {
                    # 함수를 사용하여 코드를 간결하게 해보기
                    '지점명'   :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > strong').text.strip(),
                    '주소'     : exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > p').text.strip().split('\n')[0],
                    '시도'     :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > p').text.strip().split()[0],
                    '시군구'   :  exchange_text(f'.quickSearchResultBoxSidoGugun > li:nth-child({cur_sigun+1}) > p').text.strip().split()[1],
                    '타입'     :  find_loc(tmp) # 5-3 매장의 타입이 현재 뒤죽박죽이어서 조건문을 통해서 판별, 함수 사용
                }
            )
            
            # append 후에 지역 검색 창의 스크롤을 내려줘야 정보가 온전히 담긴다.
            if cur_sigun < len(sigun) - 2:
                action = ActionChains(driver)
                scrollList = exchange_texts(f'.quickSearchResultBoxSidoGugun> li')
                action.move_to_element(scrollList[cur_sigun+2]).perform()
            else:
                 continue
            # 혹시 모를 상황에 대비해 각 구의 매장을 돌 때마다 sleep을 걸어둠
            #time.sleep(random.randint(1,2))
    
    # 각 시의 반복이 끝날때마다 마지막 값을 출력하여 잘 들어오고있는지 확인
    #print(starbucks_loc[count - 1])

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [07:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 전체 검색 옆에 있는 수들을 합계와 리스트의 길이가 동일하다면 데이터를 잘 수집했다고 볼 수 있다.
len(starbucks_loc) == count

True

In [14]:
# 창닫기
driver.close()